In [123]:
# 1.目的
# 予想脚質がゴール直近の通貨順位に対してどのくらい正確なのか調べたい
# （正確なのであれば、例えば予想脚質が"逃げ"ならゴール直前の通貨順位は小さいはず）
# 2.やったこと
# XGBoostを使う。目的変数を通過順位とし、説明変数を予想脚質として予測したときの精度を調べる
# ただし、学習データは同レースにおける馬の脚質予想の割合でフィルタリングしてから分析に用いる。
# なぜならば、脚質予想の割合が異なるレースが混じっている状態で正しく分析できないからである。
# 例えば通過順位が3位であるとしても、以下の2パターンが考えられる。
# ・予想脚質が追込である馬が8頭中6頭いて、追込の馬が後方集団にいながら通過順位が3位である
# ・予想脚質が逃げである馬が8頭中3頭いて、逃げの馬が前方集団にいながら通過順位が3位である
# 3.結果に対する所感
# 予想脚質は通過順位に有意であると考えられる。
# 特に逃げ・先行の頭数と差し・追込の頭数の差が小さいほど有意である。
# ただし逃げ・先行の頭数が差し・追込みの頭数を上回る場合においては一部有意ではない結果が得られる。

In [1]:
# インポート
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score

In [7]:
# CSVファイルの読み込み
# df = pd.read_csv('input3.csv', encoding='cp932')
df = pd.read_csv('input3.csv', encoding='utf-8')
# レースごとの馬のタイム指数の平均値を計算し、タイム指数との差を計算する
race_avg_time_index = df.groupby('race_id')['time_index'].transform('mean')
# 平均との差を新しいカラムとして追加
df['time_index_diff_from_avg'] = df['time_index'] - race_avg_time_index
# 最終コーナー通過順位と着順の差
df['position_4_diff'] = df['position_4'] - df['finish_rank']
# 同一レース内での上がり3ハロンタイムの順位（昇順＝速い順）
df['last_3f_rank'] = df.groupby('race_id')['last_3_furlongs'].rank(method='min', ascending=True).astype(int)
# style_nameはカテゴリ変数であり文字列だがそのまま使えないのでエンコーディングして数値に変換する
# 脚質を one-hot エンコーディングして、レースIDごとに集計
style_counts = pd.get_dummies(df['style_name']).groupby(df['race_id']).sum()
# 列名をわかりやすく変更
style_counts = style_counts.rename(columns={
    '逃げ': '逃げ頭数',
    '先行': '先行頭数',
    '差し': '差し頭数',
    '追込': '追込頭数'
})
# 元の df にマージ（レースIDに基づいて）
df = df.merge(style_counts, left_on='race_id', right_index=True)
# 逃げ + 先行 の合計カラムを追加
df['逃げ・先行頭数'] = df['逃げ頭数'] + df['先行頭数']
# 差し + 追込 の合計カラムを追加
df['差し・追込頭数'] = df['差し頭数'] + df['追込頭数']
# 明示的にマッピングする辞書を定義
style_mapping = {
    '逃げ': 1,
    '先行': 2,
    '差し': 3,
    '追込': 4
}
front_back_mapping = {
    '逃げ': 1,
    '先行': 1,
    '差し': 2,
    '追込': 2
}
# 新しい列 'style_encoded' を追加（元の 'style_name' 列はそのまま）
df['style_encoded'] = df['style_name'].map(style_mapping)
# 新しい列 'style_encoded' を追加（元の 'style_name' 列はそのまま）
df['front_back_encoded'] = df['style_name'].map(front_back_mapping)
# 頭数の割合でフィルタリング
number_of_horses = 8
number_of_FR = 3
number_of_RR = number_of_horses - number_of_FR
style_name = '差し'
# style_and_number_of_style_filtered_df = df[(df['逃げ・先行頭数'] == number_of_FR) & (df['差し・追込頭数'] == number_of_RR) & (df['style_encoded'] == style_mapping[style_name])]
style_and_number_of_style_filtered_df = df[(df['逃げ・先行頭数'] == number_of_FR) & (df['差し・追込頭数'] == number_of_RR)]
print(f"フィルタリング後のレコード数: {len(style_and_number_of_style_filtered_df)}")
# 結果を確認
print(style_and_number_of_style_filtered_df[[
    "race_id",
    "time_index",
    "time_index_diff_from_avg",
    "style_encoded",
    "front_back_encoded",
    "position_4",
    "finish_rank",
    "position_4_diff",
    "last_3_furlongs",
    "last_3f_rank"
]].head(8))
# レースIDの先頭2桁を抽出（文字列として扱う）
style_and_number_of_style_filtered_df.loc[:, 'race_id_year'] = style_and_number_of_style_filtered_df['race_id'].astype(str).str[:2]
# 対象：23 または 24 のレースのみ抽出
year_filtered_df = style_and_number_of_style_filtered_df[style_and_number_of_style_filtered_df['race_id_year'].isin(['23', '24'])]
# 件数カウント
count_23 = (year_filtered_df['race_id_year'] == '23').sum()
count_24 = (year_filtered_df['race_id_year'] == '24').sum()
# 割合を計算
rate_24 = count_24 / (count_23 + count_24)
# 結果表示
print(f"24の割合: {rate_24:.2f}")
# 回帰の場合→XGBRegressorを使う
# 説明変数
# x = style_and_number_of_style_filtered_df[["style_encoded", "time_index_diff_from_avg"]]
# x = style_and_number_of_style_filtered_df[["style_encoded"]]
x = style_and_number_of_style_filtered_df[["front_back_encoded", "horse_number"]]
# x = style_and_number_of_style_filtered_df[["front_back_encoded"]]
# x = style_and_number_of_style_filtered_df[["time_index_diff_from_avg"]]
# 目的変数
y1 = style_and_number_of_style_filtered_df["position_1"]
y2 = style_and_number_of_style_filtered_df["position_2"]
y3 = style_and_number_of_style_filtered_df["position_3"]
# 学習データと検証データの分割（検証データ割合はrate_24）
x_train, x_test, y1_train, y1_test = train_test_split(x, y1, train_size=rate_24, shuffle=False)
x_train, x_test, y2_train, y2_test = train_test_split(x, y2, train_size=rate_24, shuffle=False)
x_train, x_test, y3_train, y3_test = train_test_split(x, y3, train_size=rate_24, shuffle=False)
# モデルの学習
model1 = xgb.XGBRegressor(objective='reg:squarederror', max_depth=3, random_state=42)
model1.fit(x_train, y1_train)
model2 = xgb.XGBRegressor(objective='reg:squarederror', max_depth=3, random_state=42)
model2.fit(x_train, y2_train)
model3 = xgb.XGBRegressor(objective='reg:squarederror', max_depth=3, random_state=42)
model3.fit(x_train, y3_train)
# 予測
y1_pred = model1.predict(x_test)
y2_pred = model2.predict(x_test)
y3_pred = model3.predict(x_test)
# y*_predの評価
rmse1 = np.sqrt(mean_squared_error(y1_test, y1_pred))
r2_1 = r2_score(y1_test, y1_pred)
print(f"RMSE: {rmse1:.4f}")
print(f"R^2 Score: {r2_1:.4f}")
rmse2 = np.sqrt(mean_squared_error(y2_test, y2_pred))
r2_2 = r2_score(y2_test, y2_pred)
print(f"RMSE: {rmse2:.4f}")
print(f"R^2 Score: {r2_2:.4f}")
rmse3 = np.sqrt(mean_squared_error(y3_test, y3_pred))
r2_3 = r2_score(y3_test, y3_pred)
print(f"RMSE: {rmse3:.4f}")
print(f"R^2 Score: {r2_3:.4f}")
# y*_rankの評価
# 予測結果を DataFrame に変換して race_id と紐付け
y1_pred_df = pd.DataFrame({
    'race_id': style_and_number_of_style_filtered_df.loc[x_test.index, 'race_id'].values,
    'y1_pred': y1_pred
})
y2_pred_df = pd.DataFrame({
    'race_id': style_and_number_of_style_filtered_df.loc[x_test.index, 'race_id'].values,
    'y2_pred': y2_pred
})
y3_pred_df = pd.DataFrame({
    'race_id': style_and_number_of_style_filtered_df.loc[x_test.index, 'race_id'].values,
    'y3_pred': y3_pred
})
# 各レースごとに予測値の小さい順に順位を振る（昇順）
y1_pred_df['y1_rank'] = y1_pred_df.groupby('race_id')['y1_pred'].rank(method='first').astype(int)
y2_pred_df['y2_rank'] = y2_pred_df.groupby('race_id')['y2_pred'].rank(method='first').astype(int)
y3_pred_df['y3_rank'] = y3_pred_df.groupby('race_id')['y3_pred'].rank(method='first').astype(int)
# 3. 実着順を追加
y1_pred_df['y1_actual'] = y1_test.values
y2_pred_df['y2_actual'] = y2_test.values
y3_pred_df['y3_actual'] = y3_test.values
#評価
rmse_1 = np.sqrt(mean_squared_error(y1_pred_df['y1_actual'], y1_pred_df['y1_rank']))
r2_1 = r2_score(y1_pred_df['y1_actual'], y1_pred_df['y1_rank'])
print(f"RMSE: {rmse1:.4f}")
print(f"R^2 Score: {r2_1:.4f}")
rmse_2 = np.sqrt(mean_squared_error(y2_pred_df['y2_actual'], y2_pred_df['y2_rank']))
r2_2 = r2_score(y2_pred_df['y2_actual'], y2_pred_df['y2_rank'])
print(f"RMSE: {rmse2:.4f}")
print(f"R^2 Score: {r2_2:.4f}")
rmse_3 = np.sqrt(mean_squared_error(y3_pred_df['y3_actual'], y3_pred_df['y3_rank']))
r2_3 = r2_score(y3_pred_df['y3_actual'], y3_pred_df['y3_rank'])
print(f"RMSE: {rmse3:.4f}")
print(f"R^2 Score: {r2_3:.4f}")
# 実データを出力して確認
# 表示の最大行数・列数を設定（必要に応じて調整）
pd.set_option('display.max_rows', 200)   # 行を最大200行まで表示（例：100件見る用）
pd.set_option('display.max_columns', None)  # 列はすべて表示
pd.set_option('display.width', 1000)    # 横幅も広めに設定（必要に応じて調整）
pd.set_option('display.max_colwidth', None)  # 各列の内容も省略せず表示
y1_pred_df.sort_values(by=['race_id', 'y1_actual'], ascending=[True, True]).head(100)

フィルタリング後のレコード数: 1528
      race_id  time_index  time_index_diff_from_avg  style_encoded  front_back_encoded  position_4  finish_rank  position_4_diff  last_3_furlongs  last_3f_rank
64  230104K09          73                      8.75            4.0                 2.0           6            1                5             38.8             1
65  230104K09          53                    -11.25            3.0                 2.0           8            7                1             40.7             6
66  230104K09          70                      5.75            4.0                 2.0           4            4                0             39.7             3
67  230104K09          52                    -12.25            3.0                 2.0           7            8               -1             41.2             7
68  230104K09          72                      7.75            2.0                 1.0           1            2               -1             39.9             4
69  230104K09      

C:\Users\ryo\AppData\Local\Temp\ipykernel_19184\447880363.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  style_and_number_of_style_filtered_df.loc[:, 'race_id_year'] = style_and_number_of_style_filtered_df['race_id'].astype(str).str[:2]


,race_id,y1_pred,y1_rank,y1_actual
4,231205Z04,2.941387,3,1
5,231205Z04,2.833969,2,2
1,231205Z04,5.338820,6,3
0,231205Z04,2.400021,1,4
2,231205Z04,5.539788,8,4
6,231205Z04,5.438868,7,6
7,231205Z04,4.899968,4,7
3,231205Z04,5.259861,5,8
12,231210Z07,2.941387,2,1
15,231210Z07,4.899968,4,2
